In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

cuda:0


### Dataset

In [2]:
train_batch_size = 64
test_batch_size = 64

train_set = datasets.MNIST('./data',train=True, download=True, transform=transforms.ToTensor())
trainloader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)

test_set = datasets.MNIST('./data',train=False, download=True, transform=transforms.ToTensor())
testloader = DataLoader(test_set, batch_size= test_batch_size, shuffle= True)

### Define train&test accuracy

In [3]:
def train_accuracy(model):
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in trainloader:
        
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\ntrain set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct,len(trainloader.dataset),100. * correct / len(trainloader.dataset)))

In [5]:
def test_accuracy(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in testloader:
        
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct,len(testloader.dataset),100. * correct / len(testloader.dataset)))

### Define TeacherNet

In [6]:
class Teacher_Net(nn.Module):
    
    def __init__(self):
        super(Teacher_Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
    
    
model_T = Teacher_Net().to(device)

### Training TeacherNet

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_T.parameters(),lr=0.001)

def train_teacherNet(model,epoch):
    
    for i in range(epoch):
        
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            output = model_T(data)

            loss = loss_fn(output, target)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))

        torch.save(model.state_dict(), './teacher_weights.pt')

In [7]:
train_teacherNet(model_T,50)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.284817
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.267364
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.108279
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.358604
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.005211
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.087104
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.151927
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.035225
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.059634
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.021818
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.040698
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.091776
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.108671
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.007107
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.205878
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.016327
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.103745
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.048906
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.142308
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.231521
Train Epoch:

Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.000013
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.017036
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.009283
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.001021
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.000029
Train Epoch: 17 [0/60000 (0%)]	Loss: 0.000210
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.000443
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.001360
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.100384
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.000250
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.000257
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.000248
Train Epoch: 17 [44800/60000 (75%)]	Loss: 0.000047
Train Epoch: 17 [51200/60000 (85%)]	Loss: 0.000001
Train Epoch: 17 [57600/60000 (96%)]	Loss: 0.032482
Train Epoch: 18 [0/60000 (0%)]	Loss: 0.000919
Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.000036
Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.000011
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.000039
Train Epoch: 18 [25600/60000 (43%)]	Loss: 0

Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.000180
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.000088
Train Epoch: 33 [0/60000 (0%)]	Loss: 0.000024
Train Epoch: 33 [6400/60000 (11%)]	Loss: 0.000182
Train Epoch: 33 [12800/60000 (21%)]	Loss: 0.000085
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.000000
Train Epoch: 33 [25600/60000 (43%)]	Loss: 0.002568
Train Epoch: 33 [32000/60000 (53%)]	Loss: 0.000399
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.000115
Train Epoch: 33 [44800/60000 (75%)]	Loss: 0.000003
Train Epoch: 33 [51200/60000 (85%)]	Loss: 0.000108
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.000145
Train Epoch: 34 [0/60000 (0%)]	Loss: 0.000000
Train Epoch: 34 [6400/60000 (11%)]	Loss: 0.000520
Train Epoch: 34 [12800/60000 (21%)]	Loss: 0.000095
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.000019
Train Epoch: 34 [25600/60000 (43%)]	Loss: 0.000001
Train Epoch: 34 [32000/60000 (53%)]	Loss: 0.000014
Train Epoch: 34 [38400/60000 (64%)]	Loss: 0.002878
Train Epoch: 34 [44800/60000 (75%)]	Loss: 0

Train Epoch: 49 [6400/60000 (11%)]	Loss: 0.000000
Train Epoch: 49 [12800/60000 (21%)]	Loss: 0.000423
Train Epoch: 49 [19200/60000 (32%)]	Loss: 0.000044
Train Epoch: 49 [25600/60000 (43%)]	Loss: 0.068274
Train Epoch: 49 [32000/60000 (53%)]	Loss: 0.000000
Train Epoch: 49 [38400/60000 (64%)]	Loss: 0.000000
Train Epoch: 49 [44800/60000 (75%)]	Loss: 0.000000
Train Epoch: 49 [51200/60000 (85%)]	Loss: 0.000000
Train Epoch: 49 [57600/60000 (96%)]	Loss: 0.000000


In [8]:
test_accuracy(model_T)


Test set: Accuracy: 9884/10000 (98.84%)



### Define Student Model

In [7]:
class Student_Net(nn.Module):
    
    def __init__(self):
        super(Student_Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 800)
        self.fc2 = nn.Linear(800, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
model_S = Student_Net().to(device)

### Training Student Model

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_S.parameters(),lr=0.001)

def train_StudentNet(model,epoch):
    
    for i in range(epoch):
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            output = model_S(data)

            loss = loss_fn(output, target)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))

In [9]:
train_StudentNet(model_S,50)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.296674
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.228296
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.377767
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.180466
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.171490
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.099124
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.166532
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.191569
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.183768
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.175996
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.237999
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.053179
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.054901
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.241410
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.070222
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.027622
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.055509
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.044471
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.118346
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.128944
Train Epoch:

Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.000084
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.000496
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.000033
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.001971
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.000482
Train Epoch: 17 [0/60000 (0%)]	Loss: 0.000044
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.038438
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.000165
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.001759
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.021400
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.000358
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.000094
Train Epoch: 17 [44800/60000 (75%)]	Loss: 0.005858
Train Epoch: 17 [51200/60000 (85%)]	Loss: 0.000407
Train Epoch: 17 [57600/60000 (96%)]	Loss: 0.002130
Train Epoch: 18 [0/60000 (0%)]	Loss: 0.000367
Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.000090
Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.000871
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.000089
Train Epoch: 18 [25600/60000 (43%)]	Loss: 0

Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.000008
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.000141
Train Epoch: 33 [0/60000 (0%)]	Loss: 0.001118
Train Epoch: 33 [6400/60000 (11%)]	Loss: 0.000013
Train Epoch: 33 [12800/60000 (21%)]	Loss: 0.000000
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.000000
Train Epoch: 33 [25600/60000 (43%)]	Loss: 0.000452
Train Epoch: 33 [32000/60000 (53%)]	Loss: 0.000012
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.000004
Train Epoch: 33 [44800/60000 (75%)]	Loss: 0.000085
Train Epoch: 33 [51200/60000 (85%)]	Loss: 0.025074
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.001676
Train Epoch: 34 [0/60000 (0%)]	Loss: 0.001083
Train Epoch: 34 [6400/60000 (11%)]	Loss: 0.000042
Train Epoch: 34 [12800/60000 (21%)]	Loss: 0.000022
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.000024
Train Epoch: 34 [25600/60000 (43%)]	Loss: 0.000003
Train Epoch: 34 [32000/60000 (53%)]	Loss: 0.000086
Train Epoch: 34 [38400/60000 (64%)]	Loss: 0.000015
Train Epoch: 34 [44800/60000 (75%)]	Loss: 0

Train Epoch: 49 [6400/60000 (11%)]	Loss: 0.000000
Train Epoch: 49 [12800/60000 (21%)]	Loss: 0.009704
Train Epoch: 49 [19200/60000 (32%)]	Loss: 0.000000
Train Epoch: 49 [25600/60000 (43%)]	Loss: 0.000000
Train Epoch: 49 [32000/60000 (53%)]	Loss: 0.000022
Train Epoch: 49 [38400/60000 (64%)]	Loss: 0.000000
Train Epoch: 49 [44800/60000 (75%)]	Loss: 0.000002
Train Epoch: 49 [51200/60000 (85%)]	Loss: 0.000050
Train Epoch: 49 [57600/60000 (96%)]	Loss: 0.000039


In [10]:
test_accuracy(model_S)


Test set: Accuracy: 9839/10000 (98.39%)



### Knowledge Distillation on Mnist

In [17]:
model_T.load_state_dict(torch.load('./teacher_weights.pt')) # load Teacher model weigths
model_S = Student_Net().to(device)

### Define loss total for knowledge distillation

In [18]:
def loss_total(outputT, outputS, target, T, K):
    
    outputT_log = F.log_softmax(outputT/T, dim=1) 
    outputS_log = F.log_softmax(outputS/T, dim=1)

    KLDivLoss = nn.KLDivLoss(reduction='batchmean')
    loss_kd = KLDivLoss(outputS_log, outputT_log) 

    loss_ce = nn.CrossEntropyLoss()(outputS, target)

    loss_total = loss_ce* (1. - K) + loss_kd * (2.0 * T * T + K) 

    return loss_total

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_S.parameters(),lr=0.001)

def train_KD(model_S, model_T,epoch):
    
    for i in range(epoch):
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()

            outputS = model_S(data)

            outputT = model_T(data) # logit
            outputT = outputT.detach()

            loss = loss_total(outputT, outputS, target,3,0.5)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, batch_idx * len(data), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))

In [20]:
train_KD(model_S, model_T,50)

Train Epoch: 0 [0/60000 (0%)]	Loss: 1.157632
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.202889
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.138376
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.171390
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.037853
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.062305
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.123515
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.027581
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.072669
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.049219
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.036983
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.031498
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.041827
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.092501
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.049250
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.005088
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.038934
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.054995
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.011060
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.011113
Train Epoch:

Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.000679
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.000662
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.000095
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.000193
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.000014
Train Epoch: 17 [0/60000 (0%)]	Loss: 0.000020
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.007428
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.000521
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.000122
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.000262
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.000011
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.001720
Train Epoch: 17 [44800/60000 (75%)]	Loss: 0.000278
Train Epoch: 17 [51200/60000 (85%)]	Loss: 0.012240
Train Epoch: 17 [57600/60000 (96%)]	Loss: 0.000457
Train Epoch: 18 [0/60000 (0%)]	Loss: 0.011347
Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.000016
Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.000039
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.000322
Train Epoch: 18 [25600/60000 (43%)]	Loss: 0

Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.000045
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.001235
Train Epoch: 33 [0/60000 (0%)]	Loss: 0.000009
Train Epoch: 33 [6400/60000 (11%)]	Loss: 0.000012
Train Epoch: 33 [12800/60000 (21%)]	Loss: 0.000014
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.000000
Train Epoch: 33 [25600/60000 (43%)]	Loss: 0.000003
Train Epoch: 33 [32000/60000 (53%)]	Loss: 0.000046
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.000032
Train Epoch: 33 [44800/60000 (75%)]	Loss: 0.000000
Train Epoch: 33 [51200/60000 (85%)]	Loss: 0.000048
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.000011
Train Epoch: 34 [0/60000 (0%)]	Loss: 0.000020
Train Epoch: 34 [6400/60000 (11%)]	Loss: 0.000116
Train Epoch: 34 [12800/60000 (21%)]	Loss: 0.000042
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.000910
Train Epoch: 34 [25600/60000 (43%)]	Loss: 0.000000
Train Epoch: 34 [32000/60000 (53%)]	Loss: 0.000035
Train Epoch: 34 [38400/60000 (64%)]	Loss: 0.000001
Train Epoch: 34 [44800/60000 (75%)]	Loss: 0

Train Epoch: 49 [6400/60000 (11%)]	Loss: 0.000002
Train Epoch: 49 [12800/60000 (21%)]	Loss: 0.000010
Train Epoch: 49 [19200/60000 (32%)]	Loss: 0.000000
Train Epoch: 49 [25600/60000 (43%)]	Loss: 0.000007
Train Epoch: 49 [32000/60000 (53%)]	Loss: 0.000000
Train Epoch: 49 [38400/60000 (64%)]	Loss: 0.000004
Train Epoch: 49 [44800/60000 (75%)]	Loss: 0.000000
Train Epoch: 49 [51200/60000 (85%)]	Loss: 0.000155
Train Epoch: 49 [57600/60000 (96%)]	Loss: 0.000001


In [15]:
test_accuracy(model_S)


Test set: Accuracy: 9813/10000 (98.13%)

